<a href="https://colab.research.google.com/github/johnJay1745/CeDeFi/blob/main/SSETH_%26_SSLONG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd 
import numpy as np 
import numpy as np
import scipy.stats as st
import json
import requests
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import datetime
from datetime import datetime

In [ ]:
pool_ids = {
    "Uni_V3_rETH-ETH": "5685fa0d-a7cf-4b60-9b77-ff23b4583f81",
    "Convex_FRAX-USDC":"844a8a29-4653-42e6-b675-a6b43fac3678",
    "ETH_AAVE": "f2726d05-1f8d-4b9c-80e3-43d03d85d117",
    "USDC_AAVE": "a349fea4-d780-4e16-973e-70ca9b606db2"
}

In [ ]:
def get_hist_apy(pool_id):
    data = requests.get("https://yields.llama.fi/chart/" + pool_id)
    data_json = data.json() 
    l = []
    for apy in data_json["data"]:
        l.append([apy["timestamp"],apy["apy"]])
    return pd.DataFrame(l).rename(columns = {0:"timestamp",1:"annual_ROI"})

In [ ]:
def get_coin_price(coin,vs_currency,days):
    data = requests.get(
    "https://api.coingecko.com/api/v3/coins/" + 
    coin +
    "/market_chart?vs_currency=" +
    vs_currency +
    "&days=" + 
    days
    )
    price_list = list(np.array(data.json()["prices"])[:,1])
    return price_list

In [ ]:
# def get_leverage_ratio():
#     leverage_ratios = []
#     df = pd.DataFrame(get_coin_price("ethereum","USD","233"))
#     for i in range(len(df)):
#         ETH_ratio = 0.25
#         USDC_ratio = 0.5
#         ETH_ratio = ETH_ratio * (df[0][i] / df[0][0])
#         USDC_ratio = USDC_ratio * (df[0][0] / df[0][i])
#         leverage_ratios.append([ETH_ratio,USDC_ratio])
#     return (
#     pd.DataFrame(leverage_ratios).
#     rename(columns = {0:"ETH_ratio",1:"USDC_ratio"})
#     )

In [ ]:
def get_HF(HF):
    HF_list = []
    df = get_coin_price("ethereum","USD","234")
    for i in range(len(df)):
       current_SLETH_HF = (df[i] / df[0]) / (1 / HF)
       current_SSETH_HF = (df[0] / df[i]) / (1 / HF)
       HF_list.append([current_SLETH_HF,current_SSETH_HF])

    return pd.DataFrame(HF_list)

In [ ]:
def get_yield_info(pool_ids):
    df = pd.DataFrame()
    df["Time"] = get_hist_apy(pool_ids["Uni_V3_rETH-ETH"])["timestamp"]
    
    df["UNI_V3_returns"] = (
    get_hist_apy(pool_ids["Uni_V3_rETH-ETH"])["annual_ROI"] / 36500
    ) 

    df["FRAX-USDC_returns"] = (
    get_hist_apy(pool_ids["Convex_FRAX-USDC"])["annual_ROI"] / 36500
    ) 

    df["USDC_AAVE_returns"] = (
    get_hist_apy(pool_ids["USDC_AAVE"])["annual_ROI"] / 36500
    )
    df["ETH_AAVE_returns"] =  (
    get_hist_apy(pool_ids["ETH_AAVE"])["annual_ROI"] / 36500
    )

    return df

In [ ]:
def get_yield_APR(HF,pool_ids):
    df = get_yield_info(pool_ids)
    
    df["SSETH_yield_APR"] = (
    df["UNI_V3_returns"] * 
    0.25 +
    df["USDC_AAVE_returns"] -
    df["ETH_AAVE_returns"] * 
    (1 / 0.9) *
    0.5 + 
    df["FRAX-USDC_returns"] * 
    (0.5 / HF)
    ) 

    df["SLETH_yield_APR"] = (
    df["UNI_V3_returns"] * 
    0.5 +
    df["ETH_AAVE_returns"] -
    df["USDC_AAVE_returns"] * 
    (1 / 0.9) * 
    0.5
    ) 

    return df

In [ ]:
def rebalance(HF,pool_ids,strat_id):
    df = pd.DataFrame()
    realised_PnL_pct = []

    df["Time"] = get_yield_APR(HF,pool_ids)["Time"]
    df["ETH_price"] = get_coin_price("ethereum","USD","233")
    df["HF"] = get_HF(2)[strat_id]
    df["SLETH_yield_APR"] = get_yield_APR(HF,pool_ids)["SLETH_yield_APR"]
    df["SSETH_yield_APR"] = get_yield_APR(HF,pool_ids)["SSETH_yield_APR"]
    optimal_HFs = []
    for i in range(len(df)):
        optimal_HF = HF
        start_index = 0
        if strat_id == 0: 
            if df["HF"][i] <= optimal_HF / 0.875:
               strategy_return = (
               sum(df["SLETH_yield_APR"][start_index:i+1]) +
               (df["ETH_price"][i] / df["ETH_price"][start_index] - 1) *
               0.5 
               )
               start_index +=(i - start_index)
               optimal_HF = df["HF"][i] 
               optimal_HFs.append(optimal_HF)
               realised_PnL_pct.append(
               [df["Time"][i],strategy_return,df["ETH_price"][i]]
               )

            elif df["HF"][i] >= optimal_HF * 2.15:
                strategy_return = (
                sum(df["SLETH_yield_APR"][start_index:i+1]) +
                (df["ETH_price"][i] / df["ETH_price"][start_index] - 1) *
                0.5 
                )
                start_index += (i - start_index)
                optimal_HF = df["HF"][i]
                optimal_HFs.append(optimal_HF)
                realised_PnL_pct.append(
                [df["Time"][i],strategy_return,df["ETH_price"][i]]
                )
        else:
            if df["HF"][i] <= optimal_HF / 0.875:
               strategy_return = (
               sum(df["SLETH_yield_APR"][start_index:i+1]) + 
               (df["ETH_price"][start_index] / df["ETH_price"][i] - 1) *
               0.25 
               )
               start_index = (i - start_index)
               optimal_HF = df["HF"][i]
               optimal_HFs.append(optimal_HF) 
               realised_PnL_pct.append(
               [df["Time"][i],strategy_return,df["ETH_price"][i]]
               )
               
            elif df["HF"][i] >= optimal_HF * 2.15:
                strategy_return = (
                sum(df["SLETH_yield_APR"][start_index:i+1]) + 
                (df["ETH_price"][start_index] / df["ETH_price"][i] - 1) *
                0.25 
                )
                start_index +=(i - start_index)
                optimal_HF = df["HF"][i] 
                optimal_HFs.append(optimal_HF)
                realised_PnL_pct.append(
                [df["Time"][i],strategy_return,df["ETH_price"][i]]
                )
            
            
    return pd.DataFrame(realised_PnL_pct)

In [ ]:
def share_price(df):
    share_prices = []
    for i in range(len(df)):
        init_price = 1
        init_price *= (1 + df[1][i])
        share_prices.append(init_price)
    return share_prices

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(x = rebalance(2,pool_ids,1)[0], 
               y = share_price(rebalance(2,pool_ids,1)),
               name = "Position value (Short)"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x = rebalance(2,pool_ids,1)[0], 
               y = rebalance(2,pool_ids,1)[2],
               name = "ETH price"),
    secondary_y=True,
)
fig.update_layout(
    title = "SSETH return vs ETH price",
    yaxis_range = [0.5,3]
    # autosize = False,
    # width = 1000,
    # height = 500
)
fig.update_yaxes(title_text="ETH price", secondary_y = True)
fig.update_yaxes(title_text="Returns", secondary_y = False)